In [15]:
from pymongo import MongoClient
# from sshtunnel import SSHTunnelForwarder
import pymongo
from tqdm import tqdm
import os
import re

import numpy as np
import pandas as pd

import dateutil.parser

In [16]:
# 使用GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [17]:
! nvidia-smi

Sat Jul  2 16:05:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.129.06   Driver Version: 470.129.06   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 27%   35C    P8    21W / 260W |     22MiB / 11019MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 26%   

# MongoDB

In [18]:
mongoURI = "mongodb://%s:%s@%s/%s?authMechanism=SCRAM-SHA-1" % ("eva", "eva_30241", "140.117.69.70:30241", "eva")

try:
    conn = pymongo.MongoClient(mongoURI)
    db = conn.eva
    # db_de = db.patent_de
    # db_us = db.patent_us
    # db_cn = db.patent_cn
    
    db_rm_us = db.rm_patent_us
    db_rm_cn = db.rm_patent_cn
    db_rm_de = db.rm_patent_de

    db_rm_us_2020 = db.rm_patent_us_2020
    db_rm_cn_2020 = db.rm_patent_cn_2020
    db_rm_de_2020 = db.rm_patent_de_2020
    
except errors.ConnectionFailure as err:
    print(err)

In [19]:
db_rm_us_2020

Collection(Database(MongoClient(host=['140.117.69.70:30241'], document_class=dict, tz_aware=False, connect=True, authmechanism='SCRAM-SHA-1'), 'eva'), 'rm_patent_us_2020')

# 各國assignee patent申請數量

## DE

In [20]:
data_assignee = db_rm_de.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [21]:
df_family_de = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])
df_family_de.head()

,company,count
0,Intel Corp,206
1,Samsung Electronics Co Ltd,96
2,Intel IP Corp,53
3,Robert Bosch GmbH,46
4,Apple Inc,44


In [22]:
df_family_de.head(20)

,company,count
0,Intel Corp,206
1,Samsung Electronics Co Ltd,96
2,Intel IP Corp,53
3,Robert Bosch GmbH,46
4,Apple Inc,44
5,Siemens AG,28
6,Fraunhofer Gesellschaft zur Forderung der Ange...,26
7,Deutsche Telekom AG,23
8,Sony Corp,22
9,Giesecke and Devrient Mobile Security GmbH,21


In [30]:
df_family_de.company[6]

'Fraunhofer Gesellschaft zur Forderung der Angewandten Forschung eV'

## CN

In [23]:
data_assignee = db_rm_cn.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [24]:
df_family_cn = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])
df_family_cn.head()

,company,count
0,Huawei Technologies Co Ltd,1449
1,Samsung Electronics Co Ltd,747
2,Shenzhen Shenglu IoT Communication Technology ...,656
3,Qualcomm Inc,625
4,Shanghai Langbo Communication Technology Co Ltd,538


In [25]:
df_family_cn.head(20)

,company,count
0,Huawei Technologies Co Ltd,1449
1,Samsung Electronics Co Ltd,747
2,Shenzhen Shenglu IoT Communication Technology ...,656
3,Qualcomm Inc,625
4,Shanghai Langbo Communication Technology Co Ltd,538
5,Xian Irain IoT Technology Service Co Ltd,500
6,China Mobile Communications Group Co Ltd,430
7,Jiangsu IoT Research and Development Center,370
8,Chengdu Qinchuan IoT Technology Co Ltd,360
9,ZTE Corp,335


In [31]:
df_family_cn.company[2]

'Shenzhen Shenglu IoT Communication Technology Co Ltd'

## US

In [26]:
data_assignee = db_rm_us.aggregate([
    {"$unwind" : "$current_assignee"},
    {"$group" : {"_id" : "$current_assignee", "count":{"$sum":1}}},
    {"$sort": {"count":-1}}
    # {"$limit":10}
])

ls_company = []
ls_count = []
for data in data_assignee:
    #print(data["_id"])
    ls_company.append(data["_id"])
    ls_count.append(data["count"])

In [27]:
df_family_us = pd.DataFrame((zip(ls_company, ls_count)), columns = ['company', 'count'])
df_family_us.head()

,company,count
0,Qualcomm Inc,5647
1,Samsung Electronics Co Ltd,3740
2,Intel Corp,2083
3,International Business Machines Corp,1543
4,Telefonaktiebolaget LM Ericsson AB,1541


In [28]:
df_family_us.head(20)

,company,count
0,Qualcomm Inc,5647
1,Samsung Electronics Co Ltd,3740
2,Intel Corp,2083
3,International Business Machines Corp,1543
4,Telefonaktiebolaget LM Ericsson AB,1541
5,LG Electronics Inc,1443
6,Cisco Technology Inc,1053
7,Microsoft Technology Licensing LLC,723
8,Huawei Technologies Co Ltd,613
9,AT&T Intellectual Property I LP,528


In [29]:
df_family_us.company[18]

'Electronics and Telecommunications Research Institute ETRI'